In [1]:
import torch
from torch import nn
import numpy as np
import tabulate

import os 

# os.chdir('/gpfs/commons/home/tchen/loss_sub_space_geometry_project/loss-subspace-geometry/src')
os.chdir('/home/tristan/loss-subspace-geometry-project/loss-subspace-geometry/src')


from models.mlp import SubspaceNN, NN, NonLinearSubspaceNN

/home/tristan/envs/loss-subspace/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torchvision
import torchvision.transforms as transforms

In [14]:
# configs
data_dim = 784
hidden_size = 512
out_dim = 10
dropout_prob = 0.3
seed = 11202022
# device = torch.device('cuda')
device = torch.device('cuda:2')

# model_path = '/gpfs/commons/home/tchen/loss_sub_space_geometry_project/loss-subspace-geometry-save/models/subspace_vanilla_mlp_0.pt'
model_path = '/home/tristan/loss-subspace-geometry-project/loss-subspace-geometry-save/models/nonlinear_subspace_vanilla_mlp_0.pt'

curve_model = NonLinearSubspaceNN(input_dim=data_dim, 
                         hidden_dim=hidden_size, 
                         out_dim=out_dim, 
                         dropout_prob=dropout_prob, 
                         seed=seed).to(device)

for tuple in curve_model.state_dict():
    print(tuple)
print()
checkpoint = torch.load(model_path)
for tuple in checkpoint:
    print(tuple)
curve_model.load_state_dict(checkpoint, strict=False)

mlp.linear.weight
mlp.linear.bias
mlp.linear.weight_1
mlp.linear.line.linear_1.weight
mlp.linear.line.linear_1.bias
mlp.linear.line.linear_2.weight
mlp.linear.line.linear_2.bias
out.weight
out.bias
out.weight_1
out.line.linear_1.weight
out.line.linear_1.bias
out.line.linear_2.weight
out.line.linear_2.bias

mlp.linear.weight
mlp.linear.bias
mlp.linear.weight_1
out.weight
out.bias
out.weight_1


_IncompatibleKeys(missing_keys=['mlp.linear.line.linear_1.weight', 'mlp.linear.line.linear_1.bias', 'mlp.linear.line.linear_2.weight', 'mlp.linear.line.linear_2.bias', 'out.line.linear_1.weight', 'out.line.linear_1.bias', 'out.line.linear_2.weight', 'out.line.linear_2.bias'], unexpected_keys=[])

In [5]:
# more configs
curve_points = 61
grid_points = 21
margin_left = 0.2
margin_right = 0.2
margin_bottom = 0.2
margin_top = 0.2

In [6]:
curve_model

NonLinearSubspaceNN(
  (mlp): NonLinearSubspaceMLP(
    (linear): LinesNN(
      in_features=784, out_features=512, bias=True
      (line): ParameterizedSubspace(
        (linear_1): Linear(in_features=1, out_features=256, bias=True)
        (linear_2): Linear(in_features=256, out_features=512, bias=True)
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (out): LinesNN(
    in_features=512, out_features=10, bias=True
    (line): ParameterizedSubspace(
      (linear_1): Linear(in_features=1, out_features=5, bias=True)
      (linear_2): Linear(in_features=5, out_features=10, bias=True)
    )
  )
)

In [15]:
curve_parameters = list(curve_model.parameters())
w = []

w.append(np.concatenate([
        p.data.cpu().numpy().ravel() for p in [curve_parameters[0], curve_parameters[1], curve_parameters[3], curve_parameters[4]]
    ]))

w.append(np.concatenate([
        p.data.cpu().numpy().ravel() for p in [curve_parameters[2], curve_parameters[1], curve_parameters[5], curve_parameters[4]]
    ]))

In [17]:
isolated_model = NN(input_dim=data_dim, 
                         hidden_dim=hidden_size, 
                         out_dim=out_dim, 
                         dropout_prob=dropout_prob).to(device)
# isolated_checkpoint = torch.load('/gpfs/commons/home/tchen/loss_sub_space_geometry_project/loss-subspace-geometry-save/models/vanilla_mlp.pt')
isolated_checkpoint = torch.load('/home/tristan/loss-subspace-geometry-project/loss-subspace-geometry-save/models/nonlienar_subspace_vanilla_mlp_0.pt')
isolated_model.load_state_dict(isolated_checkpoint)

RuntimeError: Error(s) in loading state_dict for NN:
	Unexpected key(s) in state_dict: "mlp.linear.weight_1", "out.weight_1". 

In [8]:
isolated_model

NN(
  (mlp): MLP(
    (linear): Linear(in_features=784, out_features=512, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (out): Linear(in_features=512, out_features=10, bias=True)
)

In [9]:
w.append(np.concatenate([
        p.data.cpu().numpy().ravel() for p in list(isolated_model.parameters())
    ]))

In [27]:

# set up for grid for plane plotting

def get_xy(point, origin, vector_x, vector_y):
    return np.array([np.dot(point - origin, vector_x), np.dot(point - origin, vector_y)])


print('Weight space dimensionality: %d' % w[0].shape[0])

u = w[2] - w[0]
dx = np.linalg.norm(u)
u /= dx

v = w[1] - w[0]
v -= np.dot(u, v) * u
dy = np.linalg.norm(v)
v /= dy

bend_coordinates = np.stack(get_xy(p, w[0], u, v) for p in w)


Weight space dimensionality: 407050


/home/tristan/envs/loss-subspace/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if await self.run_code(code, result, async_=asy):


In [28]:
def get_weights(model: nn.Module, t):
    weights = []
    for module in model.modules():
        if isinstance(module, nn.Linear):
            # add attribute for weight dimensionality and subspace dimensionality
            setattr(module, f'alpha', t)
            weights.extend([module.get_weight(), module.bias.data])
        # weights.extend([w for w in module.compute_weights_t(coeffs_t) if w is not None])
    return np.concatenate([w.detach().cpu().numpy().ravel() for w in weights])


In [29]:
ts = np.linspace(0.0, 1.0, curve_points)
curve_coordinates = []
for t in np.linspace(0.0, 1.0, curve_points):
    weights = get_weights(model=curve_model, t=t)
    curve_coordinates.append(get_xy(weights, w[0], u, v))

isolated_model_weights = w[2]
curve_coordinates.append(get_xy(isolated_model_weights, w[0], u, v))
curve_coordinates = np.stack(curve_coordinates)

G = grid_points
alphas = np.linspace(0.0 - margin_left, 1.0 + margin_right, G)
betas = np.linspace(0.0 - margin_bottom, 1.0 + margin_top, G)

tr_loss = np.zeros((G, G))
tr_nll = np.zeros((G, G))
tr_acc = np.zeros((G, G))
tr_err = np.zeros((G, G))

te_loss = np.zeros((G, G))
te_nll = np.zeros((G, G))
te_acc = np.zeros((G, G))
te_err = np.zeros((G, G))

grid = np.zeros((G, G, 2))

In [33]:
# even more configs for evaluating on FashionMNIST

# data_dir = '/gpfs/commons/home/tchen/loss_sub_space_geometry_project/data/'
data_dir = '/home/tristan/loss-subspace-geometry-project/data/'
batch_size = 128

In [34]:
transform = transforms.Compose([transforms.ToTensor()])
FashionMNIST_data_train = torchvision.datasets.FashionMNIST(
    data_dir, train=True, transform=transform, download=True)

train_set, val_set = torch.utils.data.random_split(
    FashionMNIST_data_train, [50000, 10000])
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(
    val_set, batch_size=len(val_set), shuffle=False)

26422272it [00:05, 5178640.49it/s]                               


Extracting /home/tristan/loss-subspace-geometry-project/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/tristan/loss-subspace-geometry-project/data/FashionMNIST/raw



29696it [00:00, 141017.09it/s]                          


Extracting /home/tristan/loss-subspace-geometry-project/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/tristan/loss-subspace-geometry-project/data/FashionMNIST/raw



4422656it [00:01, 2387048.17it/s]                             


Extracting /home/tristan/loss-subspace-geometry-project/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/tristan/loss-subspace-geometry-project/data/FashionMNIST/raw



6144it [00:00, 33686018.01it/s]         

Extracting /home/tristan/loss-subspace-geometry-project/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/tristan/loss-subspace-geometry-project/data/FashionMNIST/raw



In [35]:
criterion = nn.CrossEntropyLoss(reduction='sum')

In [36]:
def eval(model: nn.Module, loader):
    running_loss = 0.0
    num_right = 0

    model.eval()

    for i, (x, y) in enumerate(loader):
            reshaped_x = x.reshape(x.size(0), 784)
            y_hat = model(reshaped_x.to(device))
            num_right += torch.sum(
                y.to(device) == torch.argmax(
                    y_hat, dim=-1)).detach().cpu().item()

            running_loss += criterion(y_hat, y.to(device)).item()

    return {
        'nll': running_loss / len(loader.dataset),
        'loss': running_loss / len(loader.dataset),
        'accuracy': num_right * 100.0 / len(loader.dataset),
    }

In [37]:

base_model =  NN(input_dim=data_dim, 
                         hidden_dim=hidden_size, 
                         out_dim=out_dim, 
                         dropout_prob=dropout_prob).to(device)

columns = ['X', 'Y', 'Train loss', 'Train nll', 'Train error (%)', 'Test nll', 'Test error (%)']

for i, alpha in enumerate(alphas):
    for j, beta in enumerate(betas):
        p = w[0] + alpha * dx * u + beta * dy * v

        offset = 0
        for parameter in base_model.parameters():
            size = np.prod(parameter.size())
            value = p[offset:offset+size].reshape(parameter.size())
            parameter.data.copy_(torch.from_numpy(value))
            offset += size


        # tr_res = utils.test(loaders['train'], base_model, criterion, regularizer)
        # te_res = utils.test(loaders['test'], base_model, criterion, regularizer)
        tr_res = eval(model=base_model, loader=train_loader)
        te_res = eval(model=base_model, loader=valid_loader)


        tr_loss_v, tr_nll_v, tr_acc_v = tr_res['loss'], tr_res['nll'], tr_res['accuracy']
        te_loss_v, te_nll_v, te_acc_v = te_res['loss'], te_res['nll'], te_res['accuracy']

        c = get_xy(p, w[0], u, v)
        grid[i, j] = [alpha * dx, beta * dy]

        tr_loss[i, j] = tr_loss_v
        tr_nll[i, j] = tr_nll_v
        tr_acc[i, j] = tr_acc_v
        tr_err[i, j] = 100.0 - tr_acc[i, j]

        te_loss[i, j] = te_loss_v
        te_nll[i, j] = te_nll_v
        te_acc[i, j] = te_acc_v
        te_err[i, j] = 100.0 - te_acc[i, j]

        values = [
            grid[i, j, 0], grid[i, j, 1], tr_loss[i, j], tr_nll[i, j], tr_err[i, j],
            te_nll[i, j], te_err[i, j]
        ]
        table = tabulate.tabulate([values], columns, tablefmt='simple', floatfmt='10.4f')
        if j == 0:
            table = table.split('\n')
            table = '\n'.join([table[1]] + table)
        else:
            table = table.split('\n')[2]
        print(table)

np.savez(
    os.path.join('./', 'plane.npz'),
    ts=ts,
    bend_coordinates=bend_coordinates,
    curve_coordinates=curve_coordinates,
    alphas=alphas,
    betas=betas,
    grid=grid,
    tr_loss=tr_loss,
    tr_acc=tr_acc,
    tr_nll=tr_nll,
    tr_err=tr_err,
    te_loss=te_loss,
    te_acc=te_acc,
    te_nll=te_nll,
    te_err=te_err
)

----------  ----------  ------------  -----------  -----------------  ----------  ----------------
         X           Y    Train loss    Train nll    Train error (%)    Test nll    Test error (%)
----------  ----------  ------------  -----------  -----------------  ----------  ----------------
  -19.4453    -18.9501        0.3049       0.3049            10.6940      0.3040           10.9100
  -19.4453    -12.3176        0.3145       0.3145            10.5640      0.3122           10.7700
  -19.4453     -5.6850        0.3339       0.3339            10.5940      0.3296           10.8300
  -19.4453      0.9475        0.3638       0.3638            10.8020      0.3579           11.0500
  -19.4453      7.5800        0.4059       0.4059            11.0880      0.3985           11.0700
  -19.4453     14.2126        0.4620       0.4620            11.5440      0.4531           11.5200
  -19.4453     20.8451        0.5344       0.5344            12.1020      0.5233           11.9400
  -19.4453